Idea:

Assume only few spoofing attacks

Synchronize clocks assuming all broadcast locations are correct

Estimate positions for messages based on sensor timestamps

Remove messages with high deviations between broadcast locations and estimated locations

Re-synchronize clocks and re-check broadcast locations

In [11]:
import pandas as pd
import numpy as np
import os
import re
import position_estimator
from collections import defaultdict
from tqdm import tqdm
import itertools

DATA_LOCATION = "D:/thesis_data/1hour_complete_22_01_10_05/"
DATA_LOCATION = "../raw_data/1hour/"

sensors = list()
sensors_rev = dict()
received = defaultdict(lambda: { "sensors": list() })

for file in tqdm(os.listdir(DATA_LOCATION)):
    assert re.match(r"^part-\d{5}$", file)
    with open(os.path.join(DATA_LOCATION, file), "r") as f:
        line = f.readline()
        if not line:
            continue
        assert line == "sensorType,sensorLatitude,sensorLongitude,sensorAltitude,timeAtServer,timeAtSensor,timestamp,rawMessage,sensorSerialNumber,RSSIPacket,RSSIPreamble,SNR,confidence\n"
        while (line := f.readline().strip()):
            sensorType,sensorLatitude,sensorLongitude,sensorAltitude,timeAtServer,timeAtSensor,timestamp,rawMessage,sensorSerialNumber,RSSIPacket,RSSIPreamble,SNR,confidence = line.split(',')
            
            if not position_estimator.is_relevant(rawMessage):
                continue

            if 'null' in [sensorLatitude, sensorLongitude, sensorAltitude, timeAtSensor]:
                continue
            
            sensorLatitude = float(sensorLatitude)
            sensorLongitude = float(sensorLongitude)
            sensorAltitude = float(sensorAltitude)
            timeAtSensor = float(timeAtSensor)

            if not (sensorLatitude and sensorLongitude):
                continue

            sensor = (sensorLatitude, sensorLongitude, sensorAltitude, sensorType)
            if sensor not in sensors_rev:
                sensors_rev[sensor] = len(sensors)
                sensors.append(sensor)

            if (timeAtSensor, sensors_rev[sensor]) not in received[rawMessage]["sensors"]:
                received[rawMessage]["sensors"].append((timeAtSensor, sensors_rev[sensor]))

            if not "pos" in received[rawMessage]:
                received[rawMessage]["pos"] = position_estimator.get_announced_pos(rawMessage, timeAtSensor)
    
    break # memory constraint

print("Sensors:", len(sensors))
print("Received Messages:", len(received))

  0%|          | 0/1 [00:30<?, ?it/s]

Sensors: 1364
Received Messages: 77527


In [12]:
# remove messages that have been sent more than once
print("Messages before:", len(received))
for msg in list(received.keys()):
    if len(set([s for t, s in received[msg]["sensors"]])) < len(received[msg]["sensors"]):
        del received[msg]
print("Messages after: ", len(received))

Messages before: 77527
Messages after:  74616


In [17]:
C = 299792458 # light speed, meters per second

time_delta = [ [ [] for j in range(len(sensors)) ] for i in range(len(sensors)) ]

for msg in tqdm(received):
    if not received[msg]["pos"]:
        continue

    sensor_dists_to_msg_origin = {x[1]: received[msg]["pos"].dist(position_estimator.GeoPoint(*sensors[x[1]][:3])) for x in received[msg]["sensors"]}
    time_to_sensor = { s: x / C for s, x in sensor_dists_to_msg_origin.items() }

    for (t1, s1), (t2, s2) in itertools.combinations(received[msg]["sensors"], 2):
        assert s1 != s2
        td = (t1 - time_to_sensor[s1]) - (t2 - time_to_sensor[s2])
        time_delta[s1][s2].append(td)
        time_delta[s2][s1].append(-td)

lens = []
for i in range(len(sensors)):
    for j in range(i+1, len(sensors)):
        lens.append(len(time_delta[i][j]))

np.histogram(lens)

100%|██████████| 74616/74616 [00:47<00:00, 1575.62it/s]


(array([924834,   3534,    870,    241,     51,     21,      4,      6,
             3,      2], dtype=int64),
 array([  0. ,  98.6, 197.2, 295.8, 394.4, 493. , 591.6, 690.2, 788.8,
        887.4, 986. ]))

We now have all time deltas between stations that picked up the same signal.
We model the clock shifts using gaussian distributions, with some mean mu and standard deviation sigma

Then, to get time delta probability distribution between two stations that didn't measure the same message, we convolute the probability density functions of stations with directly known time-delta

Example:
sensors A and B measured the same 100 messages, so we can directly model their time delta distribution function as: td_AB = Gauss(mu_AB, sigma_AB)
Similarly, assume we estimate the density function for sensors B and C as Gauss(mu_BC, sigma_BC)

Now, to get the density function for sensors A and C, we can convolute these two density functions:
density function of time delta A-C = Gauss(mu_AB, sigma_AB) * Gauss(mu_BC, sigma_BC) = Gauss(mu_AB + mu_BC, sqrt(sigma_AB^2 + sigma_BC^2))

We do this for all paths going from A to C, and sum up all PDFs to get the final PDF for the time delta between A and C.
For efficiency, we could just do a "shortest path", where the path lengths are the sum of variances.
This way, we just take the "best" path, with least uncertainty.

In [29]:
import statistics

time_delta_gaussians =  [ [ None for j in range(len(sensors)) ] for i in range(len(sensors)) ]

# initialize with directly known time deltas
for i in range(len(sensors)):
    for j in range(len(sensors)):
        if len(time_delta[i][j]) > 1:
            mean = statistics.mean(time_delta[i][j])
            var = statistics.variance(time_delta[i][j], xbar=mean)
            time_delta_gaussians[i][j] = (mean, var)

# floyd's algorithm
for k in tqdm(range(len(sensors))):
    for i in range(len(sensors)):
        for j in range(len(sensors)):
            if time_delta_gaussians[i][k] is not None and time_delta_gaussians[k][j] is not None:
                new_variance = time_delta_gaussians[i][k][1] + time_delta_gaussians[k][j][1]
                if time_delta_gaussians[i][j] is None or time_delta_gaussians[i][j][1] > new_variance:
                    new_mean = time_delta_gaussians[i][k][0] + time_delta_gaussians[k][j][0]
                    time_delta_gaussians[i][j] = (new_mean, new_variance)


100%|██████████| 1364/1364 [07:30<00:00,  3.03it/s]


In [36]:
variances = [ time_delta_gaussians[i][j][1] for i,j in itertools.combinations(range(len(sensors)), 2) if time_delta_gaussians[i][j] is not None ]
num_conns = [ len(time_delta[i][j]) for i,j in itertools.combinations(range(len(sensors)), 2) if len(time_delta[i][j]) > 0]

print("Mean Variance:", statistics.mean(variances))
print("Median variance:", statistics.median(variances))
print("Variance modes:", statistics.multimode(variances))

print("Mean nConnections:", statistics.mean(num_conns))
print("Median nConnections:", statistics.median(num_conns))
print("nConnections modes:", statistics.multimode(num_conns))

Mean Variance: 45.997385069043624
Median variance: 5.753407907944094e-10
Variance modes: [0.0]
Mean nConnections: 41.629132798085074
Median nConnections: 19.0
nConnections modes: [1]


Using those time deltas, we can now check the aircraft locations

In [57]:
import scipy.optimize

def estimate_position(sensor_timestamps, sensor_ids, time_delta_gaussians, sensors):
    assert len(sensor_timestamps) >= 4
    assert len(sensor_timestamps) == len(sensor_ids)

    # we want to find a position p, for which we want to minimize some objective function.
    # this objective function is subject to the time deltas,
    # and the distances from the estimated position and the sensor positions
    # also, we should consider the variances in the time delta distributions:
    # if some time delta probability distribution has very big variance,
    # we can't trust that sensor to the same degree as a sensor with a small time delta variance

    # attempt 1:
    def residual_error(p):
        p = position_estimator.GeoPoint(*p)
        #print("p:", p.lat, p.lon, p.alt)
        err = 0
        for i,j in itertools.combinations(range(len(sensor_ids)), 2):
            if time_delta_gaussians[sensor_ids[i]][sensor_ids[j]] is None:
                continue

            dist_i = p.dist(position_estimator.GeoPoint(*sensors[sensor_ids[i]][:3]))
            dist_j = p.dist(position_estimator.GeoPoint(*sensors[sensor_ids[j]][:3]))
            t_i = sensor_timestamps[i] - dist_i / C
            t_j = sensor_timestamps[j] - dist_j / C
            #print(sensor_ids[i], sensor_ids[j], t_i, t_j, time_delta_gaussians[sensor_ids[i]][sensor_ids[j]])
            e = (t_i - t_j - time_delta_gaussians[sensor_ids[i]][sensor_ids[j]][0]) ** 2 / time_delta_gaussians[sensor_ids[i]][sensor_ids[j]][1]
            err += e
        print(err)
        return err
    
    bounds = scipy.optimize.Bounds([-180, -90, 0], [180, 90, 100000])
    return position_estimator.GeoPoint(*scipy.optimize.minimize(residual_error, sensors[sensor_ids[0]][:3], method='trust-constr', bounds=bounds, options={'verbose': 1}).x)



for msg in tqdm(received):
    if len(received[msg]["sensors"]) < 4:
        continue
    
    estimated_pos = estimate_position(*zip(*received[msg]["sensors"]), time_delta_gaussians, sensors)
    received[msg]["estimated_pos"] = estimated_pos
    print("Broadcast pos:", received[msg]["pos"].lat, received[msg]["pos"].lon, received[msg]["pos"].alt)
    print("Estimated pos:", received[msg]["estimated_pos"].lat, received[msg]["estimated_pos"].lon, received[msg]["estimated_pos"].alt)
    print("Dist:", received[msg]["pos"].dist(received[msg]["estimated_pos"]))

  0%|          | 2/74616 [00:56<583:02:11, 28.13s/it]

`xtol` termination condition is satisfied.
Number of iterations: 167, function evaluations: 520, CG iterations: 322, optimality: 7.35e+10, constraint violation: 0.00e+00, execution time: 5.6e+01 s.
Broadcast pos: 52.19307 5.83303 8534.4
Estimated pos: 52.34610662383408 4.6775956559686716 8.99952920885887
Dist: 81140.81663392927


C:\Users\Lukas Baege\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
  0%|          | 3/74616 [00:57<353:28:00, 17.05s/it]

`xtol` termination condition is satisfied.
Number of iterations: 197, function evaluations: 552, CG iterations: 373, optimality: 1.84e+06, constraint violation: 0.00e+00, execution time:  1.5 s.
Broadcast pos: 39.92888 -85.03836 11574.78
Estimated pos: 33.263651671593635 -79.27330663741654 207.00105105265393
Dist: 901350.4773020697


  0%|          | 4/74616 [01:01<247:24:18, 11.94s/it]

...

`xtol` termination condition is satisfied.
Number of iterations: 177, function evaluations: 572, CG iterations: 308, optimality: 7.93e+08, constraint violation: 0.00e+00, execution time: 1.9e+01 s.
Broadcast pos: 52.63131 5.86246 3825.2400000000002
Estimated pos: 50.822188675614875 18.016270809009917 9.023971062384872
Dist: 862424.4632309743


  0%|          | 10/74616 [01:48<167:18:14,  8.07s/it]

`xtol` termination condition is satisfied.
Number of iterations: 183, function evaluations: 472, CG iterations: 364, optimality: 2.68e+05, constraint violation: 0.00e+00, execution time:  1.3 s.
Broadcast pos: 46.01661 4.07352 11887.2
Estimated pos: 48.56994288913754 2.3693294845480786 709.9890488447492
Dist: 311946.1180520712


  0%|          | 11/74616 [02:10<245:40:20, 11.85s/it]


KeyboardInterrupt: 